# Flujo de trabajo ETL

Este notebook ejecuta un proceso ETL (Extract, Transform, Load) completo, desde la extracción de datos de una base de datos, la transformación de estos datos, hasta la carga de los datos transformados en otra base de datos (AWS)

## Lectura de los archivos descargados de la página de airbnb

Analizaremos los datos de la ciudad de Paris generados hasta el 4 de septiembre de 2023.

http://insideairbnb.com/get-the-data/

### Listings
Listado detallado de los alojamientos.

### Reviews
Archivo de opiniones de los usuarios de los alojamientos de airbnb.

### Geojson
GeoJSON archivo de los barrios de la ciudad.

In [180]:
# libraries
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup # Para eliminar html de description

In [181]:
listings_detailed = pd.read_csv('../data/listings.csv.gz')
listings_detailed.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,52931,https://www.airbnb.com/rooms/52931,20230904181324,2023-09-06,previous scrape,Rental unit in Paris · ★4.65 · 2 bedrooms · 2 ...,I will be delighted to welcome you to my charm...,"In some streets, Les Épinettes has also become...",https://a0.muscache.com/pictures/miso/Hosting-...,50502817,...,4.71,4.24,4.24,7511706944917,t,382,378,4,0,1.65
1,53353,https://www.airbnb.com/rooms/53353,20230904181324,2023-09-05,city scrape,Rental unit in Paris · ★4.82 · 2 bedrooms · 3 ...,A 100 m2 Loft/apartment (with 20 m2 terrace) o...,NEIGHBORHOOD: Montmartre-Clichy-Pigalles-Les F...,https://a0.muscache.com/pictures/2a1726b5-5a09...,37051412,...,4.91,4.48,4.73,7511800114704,f,1,1,0,0,0.24
2,3109,https://www.airbnb.com/rooms/3109,20230904181324,2023-09-05,city scrape,Rental unit in Paris · ★5.0 · 1 bedroom · 1 be...,Lovely Appartment with one bedroom with a Quee...,Good restaurants<br />very close the Montparna...,https://a0.muscache.com/pictures/baeae9e2-cd53...,3631,...,5.00,5.00,5.00,7511409139079,f,1,1,0,0,0.11
3,54377,https://www.airbnb.com/rooms/54377,20230904181324,2023-09-05,city scrape,Rental unit in Paris · ★4.94 · 1 bedroom · 1 b...,We love Paris. Giving tourists the opportunity...,The district is cosmopolitan and particularly ...,https://a0.muscache.com/pictures/miso/Hosting-...,247562,...,5.00,4.57,4.88,7511800523290,f,1,0,1,0,1.19
4,5396,https://www.airbnb.com/rooms/5396,20230904181324,2023-09-05,city scrape,Rental unit in Paris · ★4.56 · Studio · 1 bed ...,"NEW SOFA-BED SINCE JUNE 2023, Please disregard...","You are within walking distance to the Louvre,...",https://a0.muscache.com/pictures/52413/f9bf76f...,7903,...,4.83,4.95,4.55,7510402838018,f,1,1,0,0,2.05


In [182]:
selected_fields = ['id',
'listing_url', # Se elimina al final
'accommodates',
'bedrooms',
'beds',
'description',
'first_review',
'host_acceptance_rate',
'host_is_superhost',
'host_response_rate',
'host_response_time',
'latitude',
'longitude',
'name',
'neighborhood_overview',
'neighbourhood',
'number_of_reviews',
'price',
'property_type',
'review_scores_accuracy',
'review_scores_checkin',
'review_scores_cleanliness',
'review_scores_communication',
'review_scores_location',
'review_scores_rating',
'review_scores_value',
'reviews_per_month',
'room_type',
'source']

df_listings_summary = listings_detailed[selected_fields]
df_listings_summary.head()

,id,listing_url,accommodates,bedrooms,beds,description,first_review,host_acceptance_rate,host_is_superhost,host_response_rate,...,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,room_type,source
0,52931,https://www.airbnb.com/rooms/52931,4,2.0,2.0,I will be delighted to welcome you to my charm...,2022-11-01,97%,f,96%,...,4.65,4.59,4.47,4.71,4.24,4.65,4.24,1.65,Entire home/apt,previous scrape
1,53353,https://www.airbnb.com/rooms/53353,6,2.0,3.0,A 100 m2 Loft/apartment (with 20 m2 terrace) o...,2010-10-05,NaN,f,NaN,...,4.85,4.97,4.85,4.91,4.48,4.82,4.73,0.24,Entire home/apt,city scrape
2,3109,https://www.airbnb.com/rooms/3109,2,1.0,1.0,Lovely Appartment with one bedroom with a Quee...,2016-12-27,100%,f,100%,...,5.00,5.00,5.00,5.00,5.00,5.00,5.00,0.11,Entire home/apt,city scrape
3,54377,https://www.airbnb.com/rooms/54377,3,NaN,1.0,We love Paris. Giving tourists the opportunity...,2016-08-27,94%,t,100%,...,4.96,4.97,4.72,5.00,4.57,4.94,4.88,1.19,Private room,city scrape
4,5396,https://www.airbnb.com/rooms/5396,2,NaN,1.0,"NEW SOFA-BED SINCE JUNE 2023, Please disregard...",2009-06-30,99%,f,100%,...,4.60,4.79,4.54,4.83,4.95,4.56,4.55,2.05,Entire home/apt,city scrape


#### Transformación para los listings:

- Unificar los campos: 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin' 'review_scores_communication', 'review_scores_location', 'review_scores_value' en un solo campo 'average_score' que resuma el score promedio.

- Remover signo de moneda del campo 'price'.

- Reemplazar en 'host_is_superhost' f con 0 y t con 1

**Quitar porcentajes y convertir a número:** Para 'host_response_rate' y 'host_acceptance_rate', quitamos el signo de porcentaje y luego convertimos a un tipo numérico

In [183]:
df_listings_summary['host_response_rate'] = df_listings_summary['host_response_rate'].str.rstrip('%').astype('float') / 100
df_listings_summary['host_acceptance_rate'] = df_listings_summary['host_acceptance_rate'].str.rstrip('%').astype('float') / 100

/var/folders/7r/4sszslkn68z61lfg2zps83wm0000gp/T/ipykernel_1564/2983579617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_listings_summary['host_response_rate'] = df_listings_summary['host_response_rate'].str.rstrip('%').astype('float') / 100
/var/folders/7r/4sszslkn68z61lfg2zps83wm0000gp/T/ipykernel_1564/2983579617.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_listings_summary['host_acceptance_rate'] = df_listings_summary['host_acceptance_rate'].str.rstrip('%').astype('float') / 100


**Convertir 'host_is_superhost' a boolean** Si 'host_is_superhost' contiene 'f' y 't', mapeamos a True y False respectivamente

In [184]:
df_listings_summary['host_is_superhost'] = df_listings_summary['host_is_superhost'].map({'f': False, 't': True})

/var/folders/7r/4sszslkn68z61lfg2zps83wm0000gp/T/ipykernel_1564/2616707048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_listings_summary['host_is_superhost'] = df_listings_summary['host_is_superhost'].map({'f': False, 't': True})


**Quitar signo de moneda en 'price':** Removemos de la columna 'price' el signo de moneda con una expresión regular y convertimos a valor numérico

In [185]:
df_listings_summary['price'] = df_listings_summary['price'].replace('[\$,]', '', regex=True).astype(float)

/var/folders/7r/4sszslkn68z61lfg2zps83wm0000gp/T/ipykernel_1564/3415240980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_listings_summary['price'] = df_listings_summary['price'].replace('[\$,]', '', regex=True).astype(float)


**Promedio de los Scores:** Unificamos los scores en un solo promedio, calculamos el promedio de las columnas:'review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value'. Para calcular 'average_score'

In [186]:
cols_scores = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
               'review_scores_checkin', 'review_scores_communication', 
               'review_scores_location', 'review_scores_value']

df_listings_summary['average_score'] = df_listings_summary[cols_scores].mean(axis=1).round(2)

/var/folders/7r/4sszslkn68z61lfg2zps83wm0000gp/T/ipykernel_1564/1860211364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_listings_summary['average_score'] = df_listings_summary[cols_scores].mean(axis=1).round(2)


**Convertir fechas a datetime**: Convertir la fecha 'first_review' a datetime

In [187]:
df_listings_summary['first_review'] = pd.to_datetime(df_listings_summary['first_review'])


/var/folders/7r/4sszslkn68z61lfg2zps83wm0000gp/T/ipykernel_1564/3358082654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_listings_summary['first_review'] = pd.to_datetime(df_listings_summary['first_review'])


In [188]:
# Lista de columnas a eliminar
cols_to_drop = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                'review_scores_checkin', 'review_scores_communication', 
                'review_scores_location', 'review_scores_value']

# Eliminar las columnas del DataFrame
df_listings_summary = df_listings_summary.drop(cols_to_drop, axis=1)

In [189]:
df_listings_summary.head()

,id,listing_url,accommodates,bedrooms,beds,description,first_review,host_acceptance_rate,host_is_superhost,host_response_rate,...,name,neighborhood_overview,neighbourhood,number_of_reviews,price,property_type,reviews_per_month,room_type,source,average_score
0,52931,https://www.airbnb.com/rooms/52931,4,2.0,2.0,I will be delighted to welcome you to my charm...,2022-11-01,0.97,False,0.96,...,Rental unit in Paris · ★4.65 · 2 bedrooms · 2 ...,"In some streets, Les Épinettes has also become...","Paris, Île-de-France, France",17,209.0,Entire rental unit,1.65,Entire home/apt,previous scrape,4.51
1,53353,https://www.airbnb.com/rooms/53353,6,2.0,3.0,A 100 m2 Loft/apartment (with 20 m2 terrace) o...,2010-10-05,NaN,False,NaN,...,Rental unit in Paris · ★4.82 · 2 bedrooms · 3 ...,NEIGHBORHOOD: Montmartre-Clichy-Pigalles-Les F...,"Paris, Île-de-France, France",37,297.0,Entire rental unit,0.24,Entire home/apt,city scrape,4.80
2,3109,https://www.airbnb.com/rooms/3109,2,1.0,1.0,Lovely Appartment with one bedroom with a Quee...,2016-12-27,1.00,False,1.00,...,Rental unit in Paris · ★5.0 · 1 bedroom · 1 be...,Good restaurants<br />very close the Montparna...,"Paris, Île-de-France, France",9,110.0,Entire rental unit,0.11,Entire home/apt,city scrape,5.00
3,54377,https://www.airbnb.com/rooms/54377,3,NaN,1.0,We love Paris. Giving tourists the opportunity...,2016-08-27,0.94,True,1.00,...,Rental unit in Paris · ★4.94 · 1 bedroom · 1 b...,The district is cosmopolitan and particularly ...,"Paris, Ile-de-France, France",102,120.0,Private room in rental unit,1.19,Private room,city scrape,4.86
4,5396,https://www.airbnb.com/rooms/5396,2,NaN,1.0,"NEW SOFA-BED SINCE JUNE 2023, Please disregard...",2009-06-30,0.99,False,1.00,...,Rental unit in Paris · ★4.56 · Studio · 1 bed ...,"You are within walking distance to the Louvre,...","Paris, Ile-de-France, France",354,140.0,Entire rental unit,2.05,Entire home/apt,city scrape,4.69


In [190]:
df_listings_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67942 entries, 0 to 67941
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     67942 non-null  int64         
 1   listing_url            67942 non-null  object        
 2   accommodates           67942 non-null  int64         
 3   bedrooms               52322 non-null  float64       
 4   beds                   67242 non-null  float64       
 5   description            67416 non-null  object        
 6   first_review           53185 non-null  datetime64[ns]
 7   host_acceptance_rate   45676 non-null  float64       
 8   host_is_superhost      66968 non-null  object        
 9   host_response_rate     42044 non-null  float64       
 10  host_response_time     42044 non-null  object        
 11  latitude               67942 non-null  float64       
 12  longitude              67942 non-null  float64       
 13  n

In [191]:
df_listings_summary.shape

(67942, 23)

In [192]:
df_listings_summary.isnull().sum()


id                           0
listing_url                  0
accommodates                 0
bedrooms                 15620
beds                       700
description                526
first_review             14757
host_acceptance_rate     22266
host_is_superhost          974
host_response_rate       25898
host_response_time       25898
latitude                     0
longitude                    0
name                         0
neighborhood_overview    29911
neighbourhood            29911
number_of_reviews            0
price                        0
property_type                0
reviews_per_month        14757
room_type                    0
source                       0
average_score            14757
dtype: int64

Análisis de los vacíos en bedrooms, puede que sea NaN si el tipo de habitación es Private room

In [193]:
filtered_df = df_listings_summary[df_listings_summary['room_type'] == 'Private room']
filtered_df.head()


,id,listing_url,accommodates,bedrooms,beds,description,first_review,host_acceptance_rate,host_is_superhost,host_response_rate,...,name,neighborhood_overview,neighbourhood,number_of_reviews,price,property_type,reviews_per_month,room_type,source,average_score
3,54377,https://www.airbnb.com/rooms/54377,3,NaN,1.0,We love Paris. Giving tourists the opportunity...,2016-08-27,0.94,True,1.0,...,Rental unit in Paris · ★4.94 · 1 bedroom · 1 b...,The district is cosmopolitan and particularly ...,"Paris, Ile-de-France, France",102,120.0,Private room in rental unit,1.19,Private room,city scrape,4.86
23,12268,https://www.airbnb.com/rooms/12268,2,NaN,1.0,"<b>The space</b><br />Chambre très claire, cal...",2010-05-28,NaN,False,NaN,...,Rental unit in Paris · 1 bedroom · 1 bed · 1 bath,NaN,NaN,1,60.0,Private room in rental unit,0.01,Private room,previous scrape,0.00
26,12887,https://www.airbnb.com/rooms/12887,2,1.0,1.0,Welcome to Paradise !<br />Central Paris with ...,2012-06-11,1.00,False,1.0,...,Rental unit in Paris · ★4.63 · 1 bedroom · 1 b...,- Many bars and restaurants<br />- nice superm...,"Paris, Île-de-France, France",79,80.0,Private room in rental unit,0.58,Private room,city scrape,4.71
31,16626,https://www.airbnb.com/rooms/16626,4,NaN,2.0,"Beautiful XV century ""hotel particulier"". This...",2010-02-28,1.00,True,1.0,...,Rental unit in Paris · ★4.90 · 1 bedroom · 2 b...,"Great situation next to Odeon, Saint Germain d...","Paris, Ile-de-France, France",143,160.0,Private room in rental unit,0.87,Private room,city scrape,4.85
50,22158,https://www.airbnb.com/rooms/22158,2,NaN,1.0,Welcome to this very cosy and quiet artist lof...,2012-11-15,0.96,True,1.0,...,Rental unit in Paris · ★4.92 · 1 bedroom · 1 b...,"The appartement is very well situated, 2 min f...","Paris, Ile-de-France, France",89,140.0,Private room in rental unit,0.68,Private room,city scrape,4.92


In [194]:
filtered_df.shape

(7959, 23)

In [195]:
filtered_df.isnull().sum()


id                          0
listing_url                 0
accommodates                0
bedrooms                 5888
beds                      169
description               141
first_review             1992
host_acceptance_rate     2909
host_is_superhost          66
host_response_rate       3280
host_response_time       3280
latitude                    0
longitude                   0
name                        0
neighborhood_overview    3774
neighbourhood            3774
number_of_reviews           0
price                       0
property_type               0
reviews_per_month        1992
room_type                   0
source                      0
average_score            1992
dtype: int64

# Manejo de vacíos

In [196]:
def fill_values(row):
    """
    Rellena los valores faltantes en las columnas 'beds' y 'bedrooms' basado en condiciones específicas.

    Parámetros:
    row (pd.Series): Una fila del DataFrame.

    Retorna:
    pd.Series: La fila con valores actualizados en 'beds' y 'bedrooms' si es necesario.
    """
    # Si 'beds' es NaN, 'bedrooms' es 1 y 'accommodates' es 2 o menos, establecer 'beds' a 1
    if pd.isna(row['beds']) and row['bedrooms'] == 1 and row['accommodates'] <= 2:
        row['beds'] = 1
    # Si 'bedrooms' es NaN y 'beds' no es NaN y es 1, establecer 'bedrooms' a 1
    if pd.isna(row['bedrooms']) and not pd.isna(row['beds']) and row['beds'] == 1:
        row['bedrooms'] = 1
    # Si 'beds' es NaN y 'accommodates' es 2 o menos, establecer 'beds' a 1
    if pd.isna(row['beds']) and not pd.isna(row['accommodates']) and row['accommodates'] <= 2:
        row['beds'] = 1
    # Si 'bedrooms' es NaN y el tipo de habitación es 'Private room', establecer 'bedrooms' a 1
    if pd.isna(row['bedrooms']) and row['room_type'] == 'Private room':
        row['bedrooms'] = 1
    # Si 'beds' es NaN y 'bedrooms' no es NaN, igualar 'beds' a 'bedrooms'
    if pd.isna(row['beds']) and not pd.isna(row['bedrooms']):
        row['beds'] = row['bedrooms']
    # Si tanto 'beds' como 'bedrooms' son NaN y 'accommodates' no es NaN, establecer 'beds' a 'accommodates'
    if pd.isna(row['beds']) and pd.isna(row['bedrooms']) and not pd.isna(row['accommodates']):
        row['beds'] = row['accommodates']
    # Si 'bedrooms' es NaN, 'beds' y 'accommodates' no son NaN, establecer 'bedrooms' a 'beds'
    if pd.isna(row['bedrooms']) and not pd.isna(row['beds']) and not pd.isna(row['accommodates']):
        row['bedrooms'] = row['beds']
    return row

In [197]:
df_listings_summary = df_listings_summary.apply(fill_values, axis=1)

In [198]:
df_listings_summary.isnull().sum()

id                           0
listing_url                  0
accommodates                 0
bedrooms                     0
beds                         0
description                526
first_review             14757
host_acceptance_rate     22266
host_is_superhost          974
host_response_rate       25898
host_response_time       25898
latitude                     0
longitude                    0
name                         0
neighborhood_overview    29911
neighbourhood            29911
number_of_reviews            0
price                        0
property_type                0
reviews_per_month        14757
room_type                    0
source                       0
average_score            14757
dtype: int64

Estimar el tiempo de actividad de la propiedad calculando el tiempo en meses desde el primer review hasta el 4 de septiembre que es la fecha de corte de la base de datos

In [199]:
df_listings_summary['first_review'] = pd.to_datetime(df_listings_summary['first_review'])

# Fecha de corte
fecha_corte = datetime(2023, 9, 4)

# Calcular la diferencia en meses
df_listings_summary['months_since_first_review'] = df_listings_summary['first_review'].apply(lambda x: (fecha_corte.year - x.year) * 12 + fecha_corte.month - x.month if pd.notna(x) else None)

In [200]:
# Eliminar las columnas del DataFrame
df_listings_summary = df_listings_summary.drop(['listing_url', 'first_review'], axis=1)

In [201]:
df_listings_summary.head()

,id,accommodates,bedrooms,beds,description,host_acceptance_rate,host_is_superhost,host_response_rate,host_response_time,latitude,...,neighborhood_overview,neighbourhood,number_of_reviews,price,property_type,reviews_per_month,room_type,source,average_score,months_since_first_review
0,52931,4,2.0,2.0,I will be delighted to welcome you to my charm...,0.97,False,0.96,within an hour,48.89613,...,"In some streets, Les Épinettes has also become...","Paris, Île-de-France, France",17,209.0,Entire rental unit,1.65,Entire home/apt,previous scrape,4.51,10.0
1,53353,6,2.0,3.0,A 100 m2 Loft/apartment (with 20 m2 terrace) o...,NaN,False,NaN,NaN,48.88985,...,NEIGHBORHOOD: Montmartre-Clichy-Pigalles-Les F...,"Paris, Île-de-France, France",37,297.0,Entire rental unit,0.24,Entire home/apt,city scrape,4.80,155.0
2,3109,2,1.0,1.0,Lovely Appartment with one bedroom with a Quee...,1.00,False,1.00,within a day,48.83191,...,Good restaurants<br />very close the Montparna...,"Paris, Île-de-France, France",9,110.0,Entire rental unit,0.11,Entire home/apt,city scrape,5.00,81.0
3,54377,3,1.0,1.0,We love Paris. Giving tourists the opportunity...,0.94,True,1.00,within a few hours,48.88927,...,The district is cosmopolitan and particularly ...,"Paris, Ile-de-France, France",102,120.0,Private room in rental unit,1.19,Private room,city scrape,4.86,85.0
4,5396,2,1.0,1.0,"NEW SOFA-BED SINCE JUNE 2023, Please disregard...",0.99,False,1.00,within an hour,48.85247,...,"You are within walking distance to the Louvre,...","Paris, Ile-de-France, France",354,140.0,Entire rental unit,2.05,Entire home/apt,city scrape,4.69,171.0


## Remover etiquetas HTML de description y neighborhood_overview

In [202]:
df_listings_summary['description_clean'] = df_listings_summary['description'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text() if pd.notna(x) else x)

/var/folders/7r/4sszslkn68z61lfg2zps83wm0000gp/T/ipykernel_1564/1110952527.py:1: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df_listings_summary['description_clean'] = df_listings_summary['description'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text() if pd.notna(x) else x)


In [205]:
df_listings_summary['neighborhood_overview_clean'] = df_listings_summary['neighborhood_overview'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text() if pd.notna(x) else x)

/var/folders/7r/4sszslkn68z61lfg2zps83wm0000gp/T/ipykernel_1564/3264588519.py:1: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  df_listings_summary['neighborhood_overview_clean'] = df_listings_summary['neighborhood_overview'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text() if pd.notna(x) else x)
/var/folders/7r/4sszslkn68z61lfg2zps83wm0000gp/T/ipykernel_1564/3264588519.py:1: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df_listings_summary['neighborhood_overview_clean'] = df_listings_summary['neighborhood_overview'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text() if pd.notna(x) else x)


In [206]:
df_listings_summary = df_listings_summary.drop(['neighborhood_overview'], axis=1)

In [203]:
df_listings_summary = df_listings_summary.drop(['description'], axis=1)

In [207]:
#Escribir archivo csv
df_listings_summary.to_csv('../data/listings_summary.csv', index=False)